In [1]:
!pip install sentence-transformers
#installing the library

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 2.1MB 6.6MB/s 
     |████████████████████████████████| 1.2MB 22.7MB/s 
     |████████████████████████████████| 870kB 36.8MB/s 
     |████████████████████████████████| 3.3MB 38.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=999a65b18b9c0afca4fd2c2f1eb7adc5dd0fb3f90a5acd2ddcedacb741381f2e
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=61ccec03bf15abd52f7a1a19bfddd8bf3a561cde07e16519d89359905327f6aa
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sentence-transformers sacremoses


In [2]:
#transformers is a prerequisite for sentence transformer
from torch import nn
from transformers import AutoModel, AutoTokenizer, AutoConfig
import json
from typing import List, Dict, Optional, Union, Tuple
import os
 
 
class Transformer(nn.Module):
    """Huggingface AutoModel to generate token embeddings.
    Loads the correct class, e.g. BERT / RoBERTa etc.
 
    :param model_name_or_path: Huggingface models name 
    :param max_seq_length: Truncate any inputs longer than max_seq_length
    :param model_args: Arguments (key, value pairs) passed to the Huggingface Transformers model
    :param cache_dir: Cache dir for Huggingface Transformers to store/load models
    :param tokenizer_args: Arguments (key, value pairs) passed to the Huggingface Tokenizer model
    :param do_lower_case: Lowercase the input
    """
    def __init__(self, model_name_or_path: str, max_seq_length: int = 128,
                 model_args: Dict = {}, cache_dir: Optional[str] = None,
                 tokenizer_args: Dict = {}, do_lower_case: Optional[bool] = None):
        super(Transformer, self).__init__()
        self.config_keys = ['max_seq_length']
        self.max_seq_length = max_seq_length
 
        if do_lower_case is not None:
            tokenizer_args['do_lower_case'] = do_lower_case
 
        config = AutoConfig.from_pretrained(model_name_or_path, **model_args, cache_dir=cache_dir)
        self.auto_model = AutoModel.from_pretrained(model_name_or_path, config=config, cache_dir=cache_dir)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, cache_dir=cache_dir, **tokenizer_args)
 
 
    def forward(self, features):
        """Returns token_embeddings, cls_token"""
        trans_features = {'input_ids': features['input_ids'], 'attention_mask': features['attention_mask']}
        if 'token_type_ids' in features:
            trans_features['token_type_ids'] = features['token_type_ids']
 
        output_states = self.auto_model(**trans_features, return_dict=False)
        output_tokens = output_states[0]
 
        cls_tokens = output_tokens[:, 0, :]  # CLS token is first token
        features.update({'token_embeddings': output_tokens, 'cls_token_embeddings': cls_tokens, 'attention_mask': features['attention_mask']})
 
        if self.auto_model.config.output_hidden_states:
            all_layer_idx = 2
            if len(output_states) < 3: #Some models only output last_hidden_states and all_hidden_states
                all_layer_idx = 1
 
            hidden_states = output_states[all_layer_idx]
            features.update({'all_layer_embeddings': hidden_states})
 
        return features
 
    def get_word_embedding_dimension(self) -> int:
        return self.auto_model.config.hidden_size
 
    def tokenize(self, texts: Union[List[str], List[Tuple[str, str]]]):
        """
        Tokenizes a text and maps tokens to token-ids
        """
        output = {}
        if isinstance(texts[0], str):
            to_tokenize = [texts]
        elif isinstance(texts[0], dict):
            to_tokenize = []
            output['text_keys'] = []
            for lookup in texts:
                text_key, text = next(iter(lookup.items()))
                to_tokenize.append(text)
                output['text_keys'].append(text_key)
            to_tokenize = [to_tokenize]
        else:
            batch1, batch2 = [], []
            for text_tuple in texts:
                batch1.append(text_tuple[0])
                batch2.append(text_tuple[1])
            to_tokenize = [batch1, batch2]
 
        output.update(self.tokenizer(*to_tokenize, padding=True, truncation='longest_first', return_tensors="pt", max_length=self.max_seq_length))
        return output
 
 
    def get_config_dict(self):
        return {key: self.__dict__[key] for key in self.config_keys}
 
    def save(self, output_path: str):
        self.auto_model.save_pretrained(output_path)
        self.tokenizer.save_pretrained(output_path)
 
        with open(os.path.join(output_path, 'sentence_bert_config.json'), 'w') as fOut:
            json.dump(self.get_config_dict(), fOut, indent=2)
 
    @staticmethod
    def load(input_path: str):
        #Old classes used other config names than 'sentence_bert_config.json'
        for config_name in ['sentence_bert_config.json', 'sentence_roberta_config.json', 'sentence_distilbert_config.json', 'sentence_camembert_config.json', 'sentence_albert_config.json', 'sentence_xlm-roberta_config.json', 'sentence_xlnet_config.json']:
            sbert_config_path = os.path.join(input_path, config_name)
            if os.path.exists(sbert_config_path):
                break
 
        with open(sbert_config_path) as fIn:
            config = json.load(fIn)
        return Transformer(model_name_or_path=input_path, **config)

In [3]:
import scipy

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [5]:
import os
import csv
import scipy

In [6]:
#mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#loading dataset
with open('/content/drive/MyDrive/Book3 - Copy.csv') as f:
    reader = csv.reader(f)
    fulldata = list(reader)
sentence_embeddings = model.encode(fulldata)







In [8]:
#@title Sematic Search Form
#enter query for search
query = 'google' #@param {type: 'string'}

queries = [query]
query_embeddings = model.encode(queries)


print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
  #setting up distances for results
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    #sorting on the basis of distance
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    #data structure of our result data
    print(type(fulldata))
    for idx, distance in results:
      #setting the threshold
      if distance<0.5:
        print(fulldata[idx], "(Cosine Score: %.4f)" % (1-distance))

Semantic Search Results




Query: google
<class 'list'>
['Google Trends', 'Google', 'https://www.npmjs.com/package/google-trends-api', 'The Google Trends API offers a layer for Google Trends data. It can be used to retrieve keyword popularity data by location, and date. This API makes use of several methods such as returning the current top 20 trending topics for a desired location (hotTrends), returning the top 30 searches of the last month (top30in30), or returning data by category (categoryTopCharts).', '2', '16', 'News & Media'] (Cosine Score: 0.9131)
['Google People API', 'Google', 'https://developers.google.com/people/', "The People API lets you list and manage the authenticated user's Contacts and retrieve profile information for authenticated users and their contacts.", '2', '6', 'Social Media & Technology'] (Cosine Score: 0.9000)
['Google+ API', 'Google', 'https://developers.google.com/+/web/', 'The Google+ API is the programming interface to Google+. This enables users to co